# Sprint 深層学習スクラッチ リカレントニューラルネットワーク

## 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。


a
t
=
x
t
⋅
W
x
+
h
t
−
1
⋅
W
h
+
B
h
t
=
t
a
n
h
(
a
t
)

a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)


x
t
 : 時刻tの入力 (batch_size, n_features)


W
x
 : 入力に対する重み (n_features, n_nodes)


h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


W
h
 : 状態に対する重み。 (n_nodes, n_nodes)


B
 : バイアス項 (n_nodes,)


初期状態 
h
0
 は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [11]:
class SimpleRNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] #パラメータ初期値設定
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        for i in range(x.shape[1]):
            at = np.dot(x[:,i,:], Wx) + np.dot(h_prev, Wh) + b #フォワードプロパゲーション
            h_next = np.tanh(at) #活性化関数
            h_prev = h_next 
        self.cache = (x, h_prev, h_next)
        return h_next

## 【問題2】小さな配列でのフォワードプロパゲーションの実験


小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。



In [9]:
import numpy as np

x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)


In [10]:
rnn = SimpleRNN(w_x,w_h,b)
h_test = rnn.forward(x,h)
h_test

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

### 正解
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
